# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
# pull in data from csv
cities_df = pd.read_csv('../output_data/cities.csv')
# drop any rows that do not have a value
cities_df.dropna(inplace = True) 
cities_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Camaná,-16.6228,-72.7111,60.76,82,58,7.07,PE,1628642395
1,Barrow,71.2906,-156.7887,37.42,78,90,10.36,US,1628642395
2,East London,-33.0153,27.9116,54.52,46,0,6.78,ZA,1628642395
3,Buchanan,5.8808,-10.0467,76.86,87,79,7.18,LR,1628642396
4,Camacha,33.0833,-16.3333,70.56,83,40,8.05,PT,1628642396
5,Cockburn Town,21.4612,-71.1419,82.85,77,0,19.73,TC,1628642396
6,Busselton,-33.6500,115.3333,50.04,73,100,10.31,AU,1628642397
7,Butaritari,3.0707,172.7902,83.26,67,10,12.12,KI,1628642397
8,Saldanha,-33.0117,17.9442,42.85,72,0,10.33,ZA,1628642397
9,Maxixe,-23.8597,35.3472,63.79,71,35,8.46,MZ,1628642398


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)
maxi = humidity.max()

# Plot Heatmap
fig = gmaps.figure(center=(35.0, -30.0), zoom_level=2.4)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=370,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
# Good conditions for golf
#
# A max temperature lower than 79 degrees but higher than 69.
# Wind speed less than 5 mph
# Humidity less than 40

cities_df.head(20)
hotel_df = cities_df[(cities_df['Max Temp'] < 89) & 
                        (cities_df['Max Temp'] > 69) & 
                        (cities_df['Wind Speed'] < 7) & 
                        (cities_df['Humidity'] < 35)
                       ].copy()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,Najrān,17.4924,44.1277,81.25,24,100,3.60,SA,1628642419
144,Sinegor'ye,62.0788,150.4770,75.78,34,98,3.47,RU,1628642445
196,Deputatsky,69.3000,139.9000,74.53,31,100,4.56,RU,1628642463
205,Aktau,43.6500,51.2000,75.78,34,7,5.79,KZ,1628642466
233,Mahdasht,35.7299,50.8111,84.42,31,0,6.91,IR,1628642475
244,Rāwah,34.4816,41.9093,88.45,25,0,5.57,IQ,1628642479
328,Batagay,67.6333,134.6333,79.14,25,91,6.98,RU,1628642507
420,Kumul,42.8000,93.4500,79.66,30,100,5.61,CN,1628642536
487,Alta Floresta,-9.8756,-56.0861,76.03,34,0,4.41,BR,1628642558


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
67,Najrān,17.4924,44.1277,81.25,24,100,3.60,SA,1628642419,
144,Sinegor'ye,62.0788,150.4770,75.78,34,98,3.47,RU,1628642445,
196,Deputatsky,69.3000,139.9000,74.53,31,100,4.56,RU,1628642463,
205,Aktau,43.6500,51.2000,75.78,34,7,5.79,KZ,1628642466,
233,Mahdasht,35.7299,50.8111,84.42,31,0,6.91,IR,1628642475,
244,Rāwah,34.4816,41.9093,88.45,25,0,5.57,IQ,1628642479,
328,Batagay,67.6333,134.6333,79.14,25,91,6.98,RU,1628642507,
420,Kumul,42.8000,93.4500,79.66,30,100,5.61,CN,1628642536,
487,Alta Floresta,-9.8756,-56.0861,76.03,34,0,4.41,BR,1628642558,


In [47]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # try-except to skip any that do not have a hotel nearby
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
67,Najrān,17.4924,44.1277,81.25,24,100,3.60,SA,1628642419,
144,Sinegor'ye,62.0788,150.4770,75.78,34,98,3.47,RU,1628642445,
196,Deputatsky,69.3000,139.9000,74.53,31,100,4.56,RU,1628642463,
205,Aktau,43.6500,51.2000,75.78,34,7,5.79,KZ,1628642466,
233,Mahdasht,35.7299,50.8111,84.42,31,0,6.91,IR,1628642475,
244,Rāwah,34.4816,41.9093,88.45,25,0,5.57,IQ,1628642479,
328,Batagay,67.6333,134.6333,79.14,25,91,6.98,RU,1628642507,
420,Kumul,42.8000,93.4500,79.66,30,100,5.61,CN,1628642536,
487,Alta Floresta,-9.8756,-56.0861,76.03,34,0,4.41,BR,1628642558,


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_layer = gmaps.marker_layer(locations, info_box_content=[f'{hotel}' for hotel in hotel_info])

fig = gmaps.figure(center=(30.0, -30.0), zoom_level=1)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig